Para muestrear la trayectoria del fotón usamos el método PDF, despejando  de la función de probabilidad dada por la ley de Beer tenemos que: $$x=\frac{ln(\xi)}{\mu_t}$$

donde $\mu_t=\mu_a+\mu_s$, $\mu_a$ y $\mu_s$  son los coeficientes de absorción  y scattering del medio y $\xi $ es un número aleatorio.

Si el fotón es dispersado, lo redirigiremos con la función de Henyey Greenstein, esta función toma un paraámetro usualmente denotado como $g$, si $g=1$ el fotón es dispersado en la dirección original, si $g=0$ significa que el medio es isotrópico y puede ser dispersado en cualquier dirección. Valores intermedios significan que el fotón puede ser dispersado en un cono de direcciones.
Usamos la siguiente fórmula:
 $$cos(\theta) = \left \{ \begin{matrix} 1-2 \xi  & \mbox{si } \mbox{ g=0} \\ \frac{1}{2g}(1+g^2)(\frac{1-g^2}{1-g+2g \xi})^2 & \mbox{si }\mbox{0<g<1}\end{matrix}\right.  $$



Para rotar el fotón usamos la siguiente transformación:

$$v_x'=v_x \ cos\theta+ \frac{sen \theta  \ (v_x v_z \ cos\phi-v_y \ sen\phi)}{\sqrt{1-v_z^2}}$$
 
$$v_y'=v_y \cos\theta +\frac{sen \theta  \ (v_y v_z \ cos\phi+ v_x \ sen\phi)}{\sqrt{1-v_z^2}}$$

$$v_z'=v_z \ cos\theta -\sqrt{1-v_z^2} \ sen\theta \ cos\phi$$

Primero hacemos la función de  Henyey Greenstein:

In [21]:
function cosθ(g)
    if g==0
        1-2*rand()
    else
        1/(2*g)*(1+g^2)*(1 - g^2) / (1-g+2*g*rand())
    end
end

cosθ (generic function with 1 method)

In [22]:
cosθ(.7)

0.9178236621263588

In [31]:
function rotacion(g,vx,vy,vz)
    ϕ=rand() 
    cos_θ=cosθ(g)
    senθ = sqrt(abs(1.0 - cos_θ^2)); 
    
    
   
    if vz == 1.0
        vx = senθ * cos(ϕ); #vx,vy,vz son los cosenos directores
            vy = senθ * sin(ϕ); 
        vz = cos_θ      
    
    
   elseif vz == -1.0
        vx = senθ * cos(ϕ); 
        vy = -senθ * sen(ϕ); 
        vz = -cos_θ; 
     
   else  
        denom = sqrt(abs(1.0 -vz^2)); 
        
        ux = senθ*vx*(vz*cos(ϕ) - vy * sin(ϕ) )/denom +vx*cos_θ
        uy = senθ*vy*(vz*cos(ϕ) +vx * sin(ϕ) )/denom +vy*cos_θ
        uz = -denom* senθ * cos(ϕ) + vz* cos_θ; 
         
        
    end


    vx,vy,vz
end

rotacion (generic function with 1 method)

In [46]:
rotacion(.75,2,3,1)

(0.894101718767718,0.31696240328440145,0.3164126283838697)

In [51]:
function mc(μ_a,μ_s,g,photons,d)
μ_t=μ_a+μ_s
    
dw=μ_a/μ_t #albedo
    Rd=0; Tt=0
    

x = 0.0;
y = 0.0;
z = 0;
w=1
    for i in 1:photons
i+=1

x = 0.0;
y = 0.0;
z = 0; 

        
w = 1.0;
  
μ_t=μ_a+μ_s
    
    vx=0
    vy=0
    vz=1
        
        
       while true #infinito 
        t = (-log((rand()+.0000000001)))/μ_t; 

            d_Boundary = 0; #la distancia a la frontera va a depender de la posición en z del fotón
            
            if vz > 0
            
            d_Boundary = (d - z)/vz; 
            
            elseif vz < 0 
                d_Boundary = -z/vz; 
                
                end #primero le da valor a la distancia a la  frontera 
            
            if t > d_Boundary # checa si el camino libre medio es mayor que la distancia a a frontera
                if vz > 0 #si la dirección es mayor a cero se transmite
                        Tt += w
                    else Rd += w #si la direc es menor que cero se refleja.
                 
                    end
                end
        
    # move
            x += t * vx; 
            y += t * vy; 
            z += t * vz; 
            
            
            w -= dw; # al paquete de fotones se le quita el albedo (la porción de fotones reflejados)
            vx,vy,vz=rotacion(g,vx,vy,vz)
        
        
            if w < 0.001 # Para terminar la simulación vemos si ya quedan muy poquitos  #Russian Roulette 
            if rand() <.9
                break; 
            end
            w /= 0.1
       end
    end
    end
    Rd,Tt
    end

mc (generic function with 3 methods)

In [52]:
mc(1,20,0.75,100,0.1)

(0,746.7619047619025)

Obtenemos más fotones transmitidos que reflejados. DE hecho no obtenemos ninguno reflejado. 

El código de la página: http://www.scratchapixel.com/lessons/mathematics-physics-for-computer-graphics/monte-carlo-methods-in-practice/monte-carlo-simulation

    double getCosTheta(const double &g) // sampling the H-G scattering phase function 
    { 
    if (g == 0) return 2 * drand48() - 1; 
    double mu = (1 - g * g) / (1 - g + 2 * g * drand48()); 
    return (1 + g * g - mu * mu) / (2.0 * g); 
        } 
 
    void spin(double &mu_x, double &mu_y, double &mu_z, const double &g) 
    { 
    double costheta = getCosTheta(g); 
    double phi = 2 * M_PI * drand48(); 
    double sintheta = sqrt(1.0 - costheta * costheta); // sin(theta) 
    double sinphi = sin(phi); 
    double cosphi = cos(phi); 
    if (mu_z == 1.0) { 
        mu_x = sintheta * cosphi; 
        mu_y = sintheta * sinphi; 
        mu_z = costheta; 
        } 
    else if (mu_z == -1.0) { 
        mu_x = sintheta * cosphi; 
        mu_y = -sintheta * sinphi; 
        mu_z = -costheta; 
        } 
    else { 
        double denom = sqrt(1.0 - mu_z * mu_z); 
        double muzcosphi = mu_z * cosphi; 
        double ux = sintheta * (mu_x * muzcosphi - mu_y * sinphi) / denom + mu_x * costheta; 
        double uy = sintheta * (mu_y * muzcosphi + mu_x * sinphi) / denom + mu_y * costheta; 
        double uz = -denom * sintheta * cosphi + mu_z * costheta; 
        mu_x = ux, mu_y = uy, mu_z = uz; 
        } 
    } 
 
    void MCSimulation() 
    { 
    // compute diffuse reflectance and transmittance
    uint32_t nphotons = 100000; 
    double scale = 1.0 / nphotons; 
    double sigma_a = 1, sigma_s = 2, sigma_t = sigma_a + sigma_s; 
    double d = 0.1, g = 0.75; 
    static const short m = 10; 
    double Rd = 0, Tt = 0; 
    for (int n = 0; n < nphotons; ++n) { 
        double w = 1; 
        double x = 0, y = 0, z = 0, mux = 0, muy = 0, muz = 1; 
        while (1) { 
            double s = -log(drand48()) / sigma_t; 
            double distToBoundary = 0; 
            if (muz > 0) distToBoundary = (d - z) / muz; 
            else if (muz < 0) distToBoundary = -z / muz; 
            if (s > distToBoundary) { 
                if (muz > 0) Tt += w; else Rd += w; 
                break; 
            } 
            // move
            x += s * mux; 
            y += s * muy; 
            z += s * muz; 
            // absorb
            double dw = sigma_a / sigma_t; 
            w -= dw; w = std::min(0.0, w); 
            if (w < 0.001) { // russian roulette test 
                if (drand48() > 1.0 / m) break; 
                else w *= m; 
                } 
            // scatter
            spin(mux, muy, muz, g); 
            } 
        } 
    printf("Rd %f Tt %f\n", Rd * scale, Tt * scale); 
    } 